In [1]:
import librosa
import librosa.display as librosa_display
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import os
import time

In [2]:
def prepare_RAVDESS_DS(path_audios):
    wav_paths, emotions, actors = [], [], []
    for path in tqdm(Path(path_audios).glob("*/*.wav")):
        name = str(path).split('/')[-1].split('.')[0]
        label = int(name.split("-")[2]) - 1  # Start emotions in 0
        actor = int(name.split("-")[-1])

        try:
            wav_paths.append(path)
            emotions.append(label)
            actors.append(actor)
        except Exception as e:
            # print(str(path), e)
            pass
        
    return wav_paths, emotions, actors

In [3]:
wav_paths, emotions, actors = prepare_RAVDESS_DS('RAVDESS')

1440it [00:00, 60777.23it/s]


In [4]:
mean_signal_length = 100000

def get_feature(paths:str, mfcc_len:int=39, flatten:bool=False):
    features = []
    pad_cnt, non_pad_cnt = 0, 0
    
    for i, path, in tqdm(enumerate(paths), desc='get features.....'):
        signal, fs = librosa.load(path)
        s_len = len(signal)
        
        if s_len < mean_signal_length:
            pad_len = mean_signal_length - s_len
            pad_rem = pad_len % 2
            pad_len //= 2
            signal = np.pad(signal, (pad_len, pad_len+pad_rem), 'constant', constant_values=0)
            pad_cnt += 1
            
        else:
            pad_len = s_len - mean_signal_length
            pad_len //= 2
            signal = signal[pad_len:pad_len + mean_signal_length]
            non_pad_cnt += 1
            
        mfcc = librosa.feature.mfcc(y=signal, sr=fs, n_mfcc=39)
        mfcc = mfcc.T
        
        features.append(mfcc)
    
    return features, pad_cnt, non_pad_cnt

In [5]:
features, pad_cnt, non_pad_cnt = get_feature(wav_paths)

get features.....: 1440it [01:38, 14.55it/s]


In [6]:
pad_cnt, non_pad_cnt

(1406, 34)

In [6]:
features[0].shape

(196, 39)

In [7]:
X = np.array(features)
y = np.array(emotions)

In [8]:
print(X.shape, y.shape)

os.makedirs('TIMNET-dataset', exist_ok=True)
with open('TIMNET-dataset/RAVDESS.npy', 'wb') as f:
    np.save(f, X)
    np.save(f, y)

(1440, 196, 39) (1440,)
